In [ ]:
pip install -qU pypdf

In [ ]:
pip install -U langchain-community

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("/content/india.pdf")
pages = []
async for page in loader.alazy_load():
    pages.append(page)

print(f"{pages[0].metadata}\n")
print(pages[0].page_content)

In [ ]:
pip install -qU langchain-text-splitters

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # NVDIA model NV-Embed-QA only support 512 chunk size 
    chunk_size=500,
    chunk_overlap=200,
)

docs = text_splitter.split_documents(pages)
print(docs)
len(docs)


In [ ]:
pip install astrapy

In [ ]:
import os
from astrapy import DataAPIClient
from astrapy.constants import VectorMetric
from astrapy.info import CollectionVectorServiceOptions
from google.colab import userdata

# Initialize the client and get a "Database" object
client = DataAPIClient(userdata.get("ASTRA_DB_APPLICATION_TOKEN"))
database = client.get_database(userdata.get("ASTRA_DB_API_ENDPOINT"))
print(f"* Database: {database.info().name}\n")

collection = database.create_collection(
    "aiLawyer", # Write any name for databse
    metric=VectorMetric.COSINE,
    service=CollectionVectorServiceOptions(
        provider="nvidia",
        model_name="NV-Embed-QA",
    ),
)
print(f"* Collection: {collection.full_name}\n")

In [ ]:
# Creating array of object ex:- [{"$vectorize": "chunk data"}] and using insert many query insert data into db
def split_text_into_chunks(text):
    chunks = []
    for chunk in docs:
        chunks.append({"$vectorize": f"{chunk}"})
        print(chunk)
    return chunks

data = split_text_into_chunks(docs)

In [ ]:
print(len(data))

In [ ]:
insertion_result = collection.insert_many(data)

In [ ]:
# Perform a similarity search
query = "what to do if some one make fasle case ?"
results = collection.find(
    sort={"$vectorize": query},
    limit=10,
    projection={"$vectorize": True},
)
print(f"Vector search results for '{query}':")


Vector search results for 'what to do if some one make fasle case ?':


In [ ]:
print(results)

Cursor("aiLawyer", idle, consumed so far: 0)


In [ ]:
for document in results:
    print("    ", document)

     {'_id': 'a68f2576-5691-4811-8f25-7656915811c1', '$vectorize': "page_content='to be decided by the Supreme Court.](2) Notwithstanding anything in article 132, any party appealing to the Supreme Court under clause (1) may urge as one of the grounds in such appeal that a substantial question of law as to the interpretation of this Constitution has been wrongly decided. (3) Notwithstanding anything in this article, no appeal shall, unless Parliament by law otherwise provides, lie to the Supreme Court from the judgment, decree or final order of one Judge of a High Court.134. Appellate jurisdiction of the Supreme Court in regard to criminal matters.—(1) An appeal shall lie to the Supreme Court from any judgment, final order or sentence in a criminal proceeding of a High Court in the territory of India if the High Court—(a) has on appeal reversed an order of acquittal of an accused person and sentenced him to death; or (b) has withdrawn for trial before itself any case from any court sub